Automatisation de la détection des différentes parties du cerveau à partir des segmentations automatiques (aseg.nii.gz)

In [2]:
import SimpleITK as sitk
import csv
from pathlib import Path

data_path = Path.cwd().parent.parent / "data"

registration_folder_path = data_path / "07_registered_aseg_results"
if not registration_folder_path.exists():
    raise FileNotFoundError(f"You must run first the script of 'aseg_t1_registration.py' to generate the registered data")

csv_folder_path = data_path / "09_label_csv"
csv_folder_path.mkdir(parents = True, exist_ok = True)

In [ ]:
"""
01 et 07 -> 09
Automatisation du script, stocker les sorties dans 09_label_csv
"""

mslesseg_path = data_path / "01_MSLesSeg_Dataset"
registration_path = data_path / "07_registered_aseg_results"

if not registration_path.exists():
    raise FileNotFoundError(
        f"You must run first the script of 'aseg_t1_registration.py' to generate the registered data"
    )

aseg_split_path_dict = {}
t1_split_path_dict = {}

for aseg_path in registration_path.rglob("*_aseg.nii.gz"):
    if len(aseg_path.stem.split("_")) == 3:
        arg = tuple(aseg_path.stem.split("_")[0:2])
    elif len(aseg_path.stem.split("_")) == 2:
        arg = aseg_path.stem.split("_")[0]
    aseg_split_path_dict[arg] = aseg_path

for t1_path in mslesseg_path.rglob("*_T1.nii.gz"):
    if len(t1_path.stem.split("_")) == 3:
        arg = tuple(t1_path.stem.split("_")[0:2])
    elif len(t1_path.stem.split("_")) == 2:
        arg = t1_path.stem.split("_")[0]
    t1_split_path_dict[arg] = t1_path

for split in aseg_split_path_dict.keys():
    if split not in t1_split_path_dict.keys():
        raise ValueError(f"Missing T1 file for key {split}")
    aseg_path = aseg_split_path_dict[split]
    t1_path = t1_split_path_dict[split]
    
    # CSV – écriture des mêmes infos
    if type(split) == tuple:
        csv_path = csv_folder_path / f"{split[0]}_{split[1]}_label.csv"
    elif type(split) == str:
        csv_path = csv_folder_path / f"{split}_label.csv"
    
    # Lecture des images
    aseg_img = sitk.ReadImage(aseg_path)
    t1_img   = sitk.ReadImage(t1_path)

    # LUT FreeSurfer ASEG
    freesurfer_aseg_labels = {
        0:  "Background",
        2:  "Left-Cerebral-White-Matter",
        3:  "Left-Cerebral-Cortex",
        4:  "Left-Lateral-Ventricle",
        5:  "Left-Inf-Lat-Vent",
        7:  "Left-Cerebellum-White-Matter",
        8:  "Left-Cerebellum-Cortex",
        10: "Left-Thalamus",
        11: "Left-Caudate",
        12: "Left-Putamen",
        13: "Left-Pallidum",
        17: "Left-Hippocampus",
        18: "Left-Amygdala",
        26: "Left-Accumbens-area",
        41: "Right-Cerebral-White-Matter",
        42: "Right-Cerebral-Cortex",
        43: "Right-Lateral-Ventricle",
        46: "Right-Cerebellum-White-Matter",
        47: "Right-Cerebellum-Cortex",
        49: "Right-Thalamus",
        50: "Right-Caudate",
        51: "Right-Putamen",
        52: "Right-Pallidum",
        53: "Right-Hippocampus",
        54: "Right-Amygdala",
        58: "Right-Accumbens-area"
    }

    def get_structure_name(label_value, parcellation="aseg"):
        if parcellation == "aseg":
            return freesurfer_aseg_labels.get(label_value, "Unknown")
        return "Unknown"


    # Sélection du label étudié
    label_value = 17  # exemple : hippocampe gauche

    binary = sitk.Equal(aseg_img, label_value)

    label_stats = sitk.LabelShapeStatisticsImageFilter()
    label_stats.Execute(binary)

    # Mesures
    volume_mm3  = label_stats.GetPhysicalSize(1)
    centroid_mm = label_stats.GetCentroid(1)
    bbox        = label_stats.GetBoundingBox(1)
    elongation  = label_stats.GetElongation(1)
    flatness    = label_stats.GetFlatness(1)

    structure_name = get_structure_name(label_value, parcellation="aseg")
    
    # PRINT
    print("=== Résultats ASEG ===")
    print(f"Label: {label_value}")
    print(f"Structure: {structure_name}")
    print(f"Volume: {volume_mm3:.1f} mm^3")
    print(f"Centroid (mm): {centroid_mm}")
    print(f"Bounding box (index, size): {bbox}")
    print(f"Elongation: {elongation:.3f}")
    print(f"Flatness: {flatness:.3f}")


    with open(csv_path, mode="w", newline="", encoding="utf-8") as csv_file:
        writer = csv.writer(csv_file)

        # En-tête
        writer.writerow([
            "label",
            "structure",
            "volume_mm3",
            "centroid_x_mm",
            "centroid_y_mm",
            "centroid_z_mm",
            "bbox_index_x",
            "bbox_index_y",
            "bbox_index_z",
            "bbox_size_x",
            "bbox_size_y",
            "bbox_size_z",
            "elongation",
            "flatness"
        ])

        # Données (copie exacte de ce qui est print)
        writer.writerow([
            label_value,
            structure_name,
            volume_mm3,
            centroid_mm[0],
            centroid_mm[1],
            centroid_mm[2],
            bbox[0],
            bbox[1],
            bbox[2],
            bbox[3],
            bbox[4],
            bbox[5],
            elongation,
            flatness
        ])

    print(f"\nCSV généré : {csv_path}")


=== Résultats ASEG ===
Label: 17
Structure: Left-Hippocampus
Volume: 7723.0 mm^3
Centroid (mm): (26.09504078725884, 18.65984720963357, -27.510682377314517)
Bounding box (index, size): (99, 84, 23, 30, 44, 49)
Elongation: 2.479
Flatness: 2.362

CSV généré : c:\Users\zhong\Documents\GitHub\SIR-project\data\09_label_csv\P10_T1_label.csv
=== Résultats ASEG ===
Label: 17
Structure: Left-Hippocampus
Volume: 10478.0 mm^3
Centroid (mm): (23.503244894063755, 11.378507348730679, -39.55344531399122)
Bounding box (index, size): (98, 87, 18, 29, 56, 38)
Elongation: 2.470
Flatness: 1.278

CSV généré : c:\Users\zhong\Documents\GitHub\SIR-project\data\09_label_csv\P10_T2_label.csv
=== Résultats ASEG ===
Label: 17
Structure: Left-Hippocampus
Volume: 8396.0 mm^3
Centroid (mm): (19.322534540257266, 28.43961410195331, -22.634349690328726)
Bounding box (index, size): (95, 81, 22, 28, 38, 59)
Elongation: 2.278
Flatness: 2.724

CSV généré : c:\Users\zhong\Documents\GitHub\SIR-project\data\09_label_csv\P11_T1